In [1]:
from scipy.stats import skew
import os
import warnings
from pathlib import Path

from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler

import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import plotly.express as px
from tqdm.autonotebook import tqdm
from IPython.display import display, HTML

# %load_ext autoreload
# %autoreload 2
np.random.seed(42)
tqdm.pandas()

/var/folders/8p/5_r5jcmx2sl549t8tgwqvr3c0000gn/T/ipykernel_16883/1107094137.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
preprocessed = Path("Data/processed/")

In [3]:
train_df = pd.read_parquet(preprocessed/"imputed_train.parquet")
test_df = pd.read_parquet(preprocessed/"imputed_test.parquet")
print(train_df.shape)
print(test_df.shape)
train_df.head()

(866, 28)
(134, 28)


,treatment_company,azimuth,md (ft),tvd (ft),operator,footage lateral length,well spacing,porpoise deviation,porpoise count,shale footage,...,s-velocity,youngs modulus,isip,breakdown pressure,pump rate,total number of stages,proppant volume,proppant fluid ratio,year,production
0,1,-14.430,14404,6437.000,1,7020.000,1152.576,34.400,9,0,...,6955.350,30.830,4491.000,6627.500,97,34,14222506.000,1.320,2016,3861.743
1,1,-16.320,10535,6035.000,1,4343.000,1852.010,6.960,3,0,...,7008.050,31.050,4176.000,7748.000,95,14,4437034.000,1.350,2013,512.233
2,8,-31.371,15111,6318.000,1,8372.000,2362.482,41.960,15,10042,...,6998.700,30.930,4650.647,6675.833,78,32,11923180.000,1.240,2013,1107.987
3,1,-48.480,12667,6714.000,1,5529.000,1052.818,10.640,3,2575,...,6877.280,30.660,4552.000,7625.250,99,36,10940004.000,1.230,2017,2943.454
4,1,-28.690,12606,6832.000,1,5121.000,902.851,46.200,6,0,...,7015.340,31.110,4939.000,7625.250,100,50,9865250.000,1.060,2017,2513.222


For the feature engineering part I will do some transformations only on the most significant variables shown in the EDA process for simplicity.

In [4]:
top_feats = ["md (ft)","footage lateral length","total number of stages", "proppant volume","youngs modulus","p-velocity"]
display(train_df[top_feats].describe().T)
train_df[top_feats].info()

,count,mean,std,min,25%,50%,75%,max
md (ft),866.000,13987.091,2606.818,8642.000,12174.750,13629.500,15414.000,27797.000
footage lateral length,866.000,6386.107,2533.968,1329.000,4666.000,5942.500,7710.250,19849.000
total number of stages,866.000,28.908,15.559,9.000,16.250,26.000,38.750,110.000
proppant volume,866.000,10541862.315,6234330.271,176096.000,6245604.750,9082286.000,13499181.750,41950368.000
youngs modulus,866.000,30.266,1.485,25.640,29.648,30.840,31.130,33.100
p-velocity,866.000,12761.496,711.953,10631.340,12199.060,13154.825,13273.490,13978.780


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 866 entries, 0 to 865
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   md (ft)                 866 non-null    int64  
 1   footage lateral length  866 non-null    float64
 2   total number of stages  866 non-null    int64  
 3   proppant volume         866 non-null    float64
 4   youngs modulus          866 non-null    float64
 5   p-velocity              866 non-null    float64
dtypes: float64(4), int64(2)
memory usage: 40.7 KB


In [5]:
def create_year_operation_aggregations(data, feat):
    data_frame = data.copy()
     # group base features
    gp_year_op = (
        data_frame.sort_values("year")
        .groupby(["year","operator"], as_index=False)
        .agg({feat: [np.min, np.max, np.std, skew]})
    )

    gp_year_op.columns = [
        "year",
        "operator",
        f"min_year_op_{feat}",
        f"max_year_op_{feat}",
        f"std_year_op_{feat}",
        f"skew_year_op_{feat}"
        
    ]
    gp_year_op = gp_year_op.fillna(0)
    data_frame = pd.merge(data_frame, gp_year_op, on=["year","operator"], how="left")
    return data_frame

def create_year_aggregations(data, feat):
    data_frame = data.copy()
     # group base features
    gp_year = (
        data_frame.sort_values("year")
        .groupby(["year"], as_index=False)
        .agg({feat: [np.min, np.max, np.std, skew]})
    )

    gp_year.columns = [
        "year",
        f"min_year_{feat}",
        f"max_year_{feat}",
        f"std_year_{feat}",
        f"skew_year_{feat}"
        
    ]
    gp_year = gp_year.fillna(0)
    data_frame = pd.merge(data_frame, gp_year, on=["year"], how="left")
    return data_frame

In [6]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    for feat in top_feats:
        train_df = create_year_operation_aggregations(train_df, feat)
        test_df = create_year_operation_aggregations(test_df, feat)
        #train_df = create_year_aggregations(train_df, feat)
        #test_df = create_year_aggregations(test_df, feat)

print(train_df.shape)
print(test_df.shape)
train_df.head()

(866, 52)
(134, 52)


,treatment_company,azimuth,md (ft),tvd (ft),operator,footage lateral length,well spacing,porpoise deviation,porpoise count,shale footage,...,std_year_op_proppant volume,skew_year_op_proppant volume,min_year_op_youngs modulus,max_year_op_youngs modulus,std_year_op_youngs modulus,skew_year_op_youngs modulus,min_year_op_p-velocity,max_year_op_p-velocity,std_year_op_p-velocity,skew_year_op_p-velocity
0,1,-14.430,14404,6437.000,1,7020.000,1152.576,34.400,9,0,...,4264601.167,0.499,28.120,31.170,1.397,0.017,12290.630,13595.400,546.589,0.335
1,1,-16.320,10535,6035.000,1,4343.000,1852.010,6.960,3,0,...,1633163.174,1.734,25.930,31.070,2.142,-0.512,12070.110,13593.930,536.128,-0.843
2,8,-31.371,15111,6318.000,1,8372.000,2362.482,41.960,15,10042,...,1633163.174,1.734,25.930,31.070,2.142,-0.512,12070.110,13593.930,536.128,-0.843
3,1,-48.480,12667,6714.000,1,5529.000,1052.818,10.640,3,2575,...,4406462.698,2.322,28.310,31.140,1.080,-1.225,12230.700,13591.620,412.831,-0.994
4,1,-28.690,12606,6832.000,1,5121.000,902.851,46.200,6,0,...,4406462.698,2.322,28.310,31.140,1.080,-1.225,12230.700,13591.620,412.831,-0.994


# Save to disk

In [7]:
train_df.to_parquet("Data/processed/engineered_train.parquet")
test_df.to_parquet("Data/processed/engineered_test.parquet")